In [20]:
import math
import random
import numpy as np
import pandas as pd
import nltk
nltk.download('punkt')
nltk.download('punkt_tab')
nltk.data.path.append('.')
import polars as pl
from nltk.tokenize import RegexpTokenizer
import pickle
import gzip


[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\aydin.firdouzov\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package punkt_tab to
[nltk_data]     C:\Users\aydin.firdouzov\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt_tab is already up-to-date!


In [21]:


df = pl.read_parquet('hf://datasets/LocalDoc/news_azerbaijan/data/train-*.parquet')
df = df.to_pandas()
df.head()

,date,id,title,text
0,2013-10-02,23,Rusiya Gürcüstan torpaqlarını ələ keçirmək ist...,Gürcüstan prezidenti Mixail Saakaşvili Rusiyay...
1,2013-10-02,24,Netanyahu İran prezidentini “qurd” adlandırdı,"""Axar.az"" saytının İsrail mətbuatına istinadən..."
2,2013-10-03,30,Kişilərə yaşamağa imkan verməyən qadınlar imiş,Bu gün dünyanın aparıcı ölkələrində insanların...
3,2013-10-18,337,Hansı məşhur neçə kilodur?,Mətbuatın daim diqqət mərkəzində olan ulduzlar...
4,2013-10-18,340,2014-cü ilin saç modelləri,2014-cü ilin payız-qış sezonunun ən dəbli saç ...


In [22]:
dataset = df['text'].values
dataset[:3]

array(['Gürcüstan prezidenti Mixail Saakaşvili Rusiyaya qarşı daha bir kəskin ittiham səsləndirib. Gürcüstanın "Apsyn.ge" saytının verdiyi məlumata görə, Saakaşvili bildirib ki, Bakı-Supsa nef kəməri vasitəsi ilə Rusiya Gürcüstan torpaqlarını ələ keçirmək istəyir: "Artıq xeyli torpaq sahəsi ələ keçirilib. Onların məqsədi Bakı-Supsa strateji neft kəmərini də tikanlı məftillərlə əhatə etməkdir. Hamımız özümüzü ələ almalıyıq. İndi bölücülük, kin, xal qazanmaq vaxtı deyil. Gürcüstanın məqsədi torpaqlarımızın təhlükəli ilhaqını birdəfəlik dayandırmaqdır". "Axar.az" saytı olaraq bu barədə vaxtilə Rusiyada səfir kimi fəaliyyət göstərmiş politoloq Hikmət Hacızadədən münasibət öyrəndik. Hikmət bəy məsələni belə bir lakonik fikirlə ifadə etdi: "Rusiya yenə öz ampluasındadır. Demək istəyir ki, harada biz varıq, ora bizimkidir. Harada neft varsa, ora da bizimdir. Təxminən belə bir lotuluq etmək istəyir".',
       '"Axar.az" saytının İsrail mətbuatına istinadən verdiyi məlumata görə, İsrailin Baş N

In [25]:
def lower(text: list[str]) -> list[str]:
  """
  Lowerize and tokenize the documents

  :param text: list of documents
  """
  lower_text = text.lower()
  tokenizer = RegexpTokenizer(r'\w+')
  lower_text = tokenizer.tokenize(lower_text)
  return lower_text

In [26]:
dataset = dataset[dataset != None]
train_data = []
for i,text in enumerate(dataset):
  train_data.append(lower(text))

In [27]:
def word_count(data: list[str]) -> dict:
  """
  Word counter by documents

  :param data: tokenized document dataset
  """
  word_counts = {}
  for text in data:
    for word in text:
      if word not in word_counts:
        word_counts[word] = 1
      else:
        word_counts[word] += 1
  return word_counts

In [28]:
train_data_count = word_count(train_data)

In [29]:
train_data_count = {k: v for k,v in train_data_count.items() if k not in (',',"''","”","``",'""','"',"'","'")}

In [30]:
def threshold_count(count: dict,threshold : int) -> dict:
  """
  Filter the counter dictionary with threshold
  
  :param count: word counter dictionary
  :param threshold: threshold value for filtering the dictionary
  """
  word_count = {}
  for word,count in count.items():
    if count>threshold:
      word_count[word] = count
  return word_count

In [31]:
train_data_threshold_count = threshold_count(train_data_count,300)

In [32]:
def unk_words(test_data: dict,vocab: list[str],unk_token='<UNK>') -> dict:  
  """
  Assign Unkown tag to words which cann't pass the threshold filter

  :param test_data: Test dataset for applying tag
  :param vocab: Vocabulary of the data
  :param unk_token: String variable of unknown tag
  """
  for text in test_data:
    for i,word in enumerate(text):
      if word not in vocab:
        text[i] = unk_token
  return test_data

In [33]:
train_data_unk = unk_words(train_data,train_data_threshold_count.keys())

In [34]:
def define_ngrams(data: dict,n: int,start_token='<s>',end_token='</s>') -> dict:
  """
  Function helps to create n grams

  :param data: unknown tagged dataset
  :param n: n-gram's n
  :param start_token: start token for each document
  :param end_token: end token for each document 
  """
  ngram_count = {}
  for text in data:
    text = [start_token]*n + text + [end_token]
    for word in range(len(text)-n+1):
      ngram = tuple(text[word:word+n])
      ngram_count[ngram] = ngram_count.get(ngram,0)+1
  return ngram_count

In [38]:
train_2gram_count = define_ngrams(train_data_unk,2)

In [39]:
train_3gram_count = define_ngrams(train_data_unk,3)

In [40]:
with gzip.open('bigram.pkl.gz', 'wb') as f:
  pickle.dump(train_2gram_count, f)

In [41]:
with gzip.open('trigram.pkl.gz', 'wb') as f:
  pickle.dump(train_3gram_count, f)